In [ ]:
pip install cvlib

In [ ]:
# face detection on image input
# import packages
import cvlib as cv
import sys
import cv2
import os 
from cvlib.object_detection import draw_bbox
from google.colab.patches import cv2_imshow

# read input image
image = cv2.imread('/content/drive/My Drive/Colab Notebooks/people.jpg')

# apply face detection
faces, confidences = cv.detect_face(image)

##print(faces)
##print(confidences)

# loop through detected faces
for face,conf in zip(faces,confidences):

    (startX,startY) = face[0],face[1]
    (endX,endY) = face[2],face[3]

    # draw rectangle over face
    cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

bbox, label, conf = cv.detect_common_objects(image)
output_image = draw_bbox(image, bbox, label, conf)
# display output
# press any key to close window           
cv2_imshow( image)
cv2.waitKey()

# save output
cv2.imwrite("face_detection.jpg", image)

print("Number of persons in the image is "+ str(label.count('person')))

In [ ]:
# gender detection on image
# import required packages

import cv2
import cvlib as cv
import sys
import numpy as np
from google.colab.patches import cv2_imshow

# read input image
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/people.jpg')

# apply face detection
face, conf = cv.detect_face(img)

padding = 20

# loop through detected faces
for f in face:

    (startX,startY) = max(0, f[0]-padding), max(0, f[1]-padding)
    (endX,endY) = min(img.shape[1]-1, f[2]+padding), min(img.shape[0]-1, f[3]+padding)
    
    # draw rectangle over face
    cv2.rectangle(img, (startX,startY), (endX,endY), (0,255,0), 2)

    face_crop = np.copy(img[startY:endY, startX:endX])

    # apply gender detection
    (label, confidence) = cv.detect_gender(face_crop)

    #print(confidence)
    #print(label)

    idx = np.argmax(confidence)
    label = label[idx]

    label = "{}: {:.2f}%".format(label, confidence[idx] * 100)

    Y = startY - 10 if startY - 10 > 10 else startY + 10

    cv2.putText(img, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2)

# display output
# press any key to close window           
cv2_imshow(img)

# save output
#cv2.imwrite("gender_detection.jpg", img)

In [ ]:
# gender detection on video 
# import necessary packages
import cvlib as cv
import cv2
import numpy as np
import sys
from google.colab.patches import cv2_imshow

# open video file
webcam = cv2.VideoCapture('/content/drive/My Drive/Crowd Check/street.mp4')

if not webcam.isOpened():
    print("Could not open video file")
    exit()
    
padding = 20

# loop through frames
while webcam.isOpened():

    # read frame from video 
    status, frame = webcam.read()

    if not status:
        print("Could not read frame")
        exit()

    # apply face detection
    face, confidence = cv.detect_face(frame)

    print(face)
    print(confidence)

    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX,startY) = max(0, f[0]-padding), max(0, f[1]-padding)
        (endX,endY) = min(frame.shape[1]-1, f[2]+padding), min(frame.shape[0]-1, f[3]+padding)
    
        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        face_crop = np.copy(frame[startY:endY, startX:endX])

        # apply face detection    
        (label, confidence) = cv.detect_gender(face_crop, model= 'yolov3-tiny')

        print(confidence)
        print(label)

        idx = np.argmax(confidence)
        label = label[idx]

        label = "{}: {:.2f}%".format(label, confidence[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write detected gender and confidence percentage on top of face rectangle
        cv2.putText(frame, label, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                    (0,255,0), 2)

    # display output
    cv2_imshow(frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
    
# release resources
webcam.release()
cv2.destroyAllWindows() 